# Протоколы аутентификации с нулевым разглашением

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел
from Crypto.Util import number

# КГПСЧ
from Crypto.Random import random

# Понадобится в протоколе Шнорра
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Hash import SHA256

# Немного математики
import gmpy2

# Для генерации идентификаторов клиентов
import uuid

---

In [4]:
def randint(a, b) -> gmpy2.mpz:
    """
    Генерация случайного числа с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_a = int(a)
    native_b = int(b)
    
    return gmpy2.mpz(random.randint(native_a, native_b))

def getrandbits(n) -> gmpy2.mpz:
    """
    """
    
    native_n = int(n)
    
    return gmpy2.mpz(random.getrandbits(native_n))

----

## Протокол Фиата-Шамира
![Fiat-Shamir Protocol](./images/Fiat-Shamir_2.png)

In [5]:
class FiatShamirCA(object):
    PRIME_BITS = 1024
    
    def __init__(self):
        """
        Инициализация доверенного центра.
        Генерируются два простых числа и их произведение.
        """
        
        self._db = {}
        
        self._p = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._q = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._modulus = gmpy2.mul(self._p, self._q)
        
        trace('[FiatShamirCA]', f'Modulus = {self._modulus}')
        
        
    def register_client(self, client_id: uuid.UUID, client_public_key: gmpy2.mpz) -> None:
        """
        Регистрация клиента. На вход принимает идентификатор клиента и его открытый ключ.
        Ничего не возвращает.
        Выбрасывает исключение, если клиент уже существует.
        """
        
        trace('[FiatShamirCA]', f'''Attempting to register client {client_id} 
        with public key {client_public_key}''')
        
        if client_id in self._db:
            trace('[FiatShamirCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Теперь сохраняю открытый ключ
        #
        
        self._db[client_id] = client_public_key
        trace('[FiatShamirCA]', 'Client registered successfully')
        
        
    def get_public_key(self, client_id: uuid.UUID) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента. Получает на вход идентификатор клиента.
        Возвращает открытый ключ.
        Выбрасывает исключение, если пользователь отсутствует.
        """
        
        return self._db[client_id]
       
        
    @property
    def modulus(self) -> gmpy2.mpz:
        """
        Возвращает модуль.
        """
        
        return self._modulus

In [6]:
class FiatShamirClient(object):
    def __init__(self, modulus: gmpy2.mpz, *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz] = None):
        """
        Инициализация клиента с выработкой ключевой пары и идентификатора.
        Последний параметр требуются для демонстрации неудачной аутентификации.
        """
        
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        self._r = None
        
        self._private_key = FiatShamirClient._generate_coprime(modulus)
        self._public_key = gmpy2.powmod(self._private_key, 2, modulus) if fake_data is None else fake_data[1]
        
        trace('[FiatShamirClient]', f'Client {self._id}')
        trace('[FiatShamirClient]', f'Public key = {self._public_key}')
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора.
        """
        
        return self._id
    
    
    def get_x(self, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение случайного значения, вычисляемого по формуле:
                    x = r ** 2 mod n,
        где n - модуль, r - случайное целое число из отрезка [1, n - 1].
        Значение r сохраняется.
        """
        
        self._r = randint(1, modulus - 1)
        return gmpy2.powmod(self._r, 2, modulus)
    
    
    def get_y(self, e: gmpy2.mpz, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение числа, вычисляемого по формуле:
                    y = r * s ** e mod n,
        где n - модуль, s - закрытый ключ пользователя, r - сгенерированное 
        функцией get_x внутреннее значение, e - целое число из отрезка [0, 1],
        пришедшее от проверяющего.
        """
        
        product = gmpy2.mul(self._r, gmpy2.powmod(self._private_key, e, modulus))
        return gmpy2.t_mod(product, modulus)
        
        
    @staticmethod
    def _generate_coprime(modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Генерация случайного числа из отрезка [1, n - 1], взаимно
        простого с n, где n - модуль.
        """
        
        #
        # Генерирую число до тех пор, пока оно не станет взаимно простым с модулем
        #
        
        result = randint(1, modulus - 1)
        while gmpy2.gcd(result, modulus) != 1:
            result = randint(1, modulus - 1)
            
        return result

In [7]:
class FiatShamirVerifier(object):
    def __init__(self, iterations: int):
        """
        Инициализация проверяющего. Задает коичество итераций проверки.
        """
        
        self._iterations = iterations
        
        
    def authenticate(self, client: FiatShamirClient, ca: FiatShamirCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        trace('[FiatShamirVerifier]', f'Attempt to authenticate client {client.identifier}')
        trace('[FiatShamirVerifier]', f'Number of iterations = {self._iterations}')
        
        is_successful = True
        
        for i in range(self._iterations):
            x = client.get_x(ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {x = }')
            
            e = randint(0, 1)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {e = }')
            
            y = client.get_y(e, ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {y = }')
            
            pk = ca.get_public_key(client.identifier)
            product = gmpy2.mul(x, gmpy2.powmod(pk, e, ca.modulus))
            
            successful_iteration = gmpy2.powmod(y, 2, ca.modulus) == gmpy2.t_mod(product, ca.modulus)
            is_successful &= successful_iteration
            
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] success: {successful_iteration}')
            
            if not successful_iteration:
                break
            
        print(f'Authentication successful: {is_successful}')
        return is_successful

----

In [8]:
# Создаю доверенный центр
ca = FiatShamirCA()

[TRACE] [FiatShamirCA] Modulus = 13579295163433925676067440187812410519203052923094271050329773726476126200455909408985030550749029318837763790209287614423501273844678290034769039240587986357172884600724638332957429000428162974660364548961789512166807993137848654191766296870128280587098749442839253583503325822500551429692022161875596998555226933322733614258215110679368487958096065124934234392800890701931333073453794851480176865979342465686075008397135777375836729260746396472239940873600668765295622416857163982962684213729782241635442534811720703103668833971948938189462761366663785980052895187580763760230649675056084326068025826241703146140173


In [9]:
# Создаю двух пользователей:
# - Алису - валидного пользователя
# - Еву - пользователя, представляющегося Алисой. но не знающего закрытого ключа Алисы
alice = FiatShamirClient(ca.modulus)
eve = FiatShamirClient(ca.modulus, 
                       fake_data=(alice.identifier, alice.public_key))

[TRACE] [FiatShamirClient] Client 5fc2a487-f351-45cb-95c3-04ab63f0c295
[TRACE] [FiatShamirClient] Public key = 4222514009043331730693264874083502219187494057551557383051473686436706905712595109116874721646638182829159231905478434966818342216424281815483026381182718766242914866842977532859760696580233833494392056663543345783356817426447196847860884924801077375797846595513875243727663238827685209117407101777266730211251062322977150337171176449736485551899200213882354215371233010016250235763293304134552992692067869152319730606611285868285824976368125883288242330619418540715038163360257158789448992405028223385051698864600100347469651455983792223632368653627365631038422656593470564536728716258605950430631941425659258438221
[TRACE] [FiatShamirClient] Client 5fc2a487-f351-45cb-95c3-04ab63f0c295
[TRACE] [FiatShamirClient] Public key = 42225140090433317306932648740835022191874940575515573830514736864367069057125951091168747216466381828291592319054784349668183422164242818154830263811827187662429

In [10]:
# Регистрирую валидного пользователя
ca.register_client(alice.identifier, alice.public_key)

[TRACE] [FiatShamirCA] Attempting to register client 5fc2a487-f351-45cb-95c3-04ab63f0c295 
        with public key 4222514009043331730693264874083502219187494057551557383051473686436706905712595109116874721646638182829159231905478434966818342216424281815483026381182718766242914866842977532859760696580233833494392056663543345783356817426447196847860884924801077375797846595513875243727663238827685209117407101777266730211251062322977150337171176449736485551899200213882354215371233010016250235763293304134552992692067869152319730606611285868285824976368125883288242330619418540715038163360257158789448992405028223385051698864600100347469651455983792223632368653627365631038422656593470564536728716258605950430631941425659258438221
[TRACE] [FiatShamirCA] Client registered successfully


In [11]:
# Для простоты будет 3 итерации
ITERATIONS = 3

# Создаю проверяющую сторону
verifier = FiatShamirVerifier(ITERATIONS)

In [12]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 5fc2a487-f351-45cb-95c3-04ab63f0c295
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(10642514515741024778152952283363369918209799677019657984634218994330096212139955444071220742567633927660636820216240631775042388396325245892175440145095734812912175059417975348883604912604173692363772379472523708735277686599716788704768202848065923471462771204210600198991930633579488737513809584327055617188682471248580027992000942619798395367080830428989124914429743972794597406566144616773554127181638440783881452090368309583936717532285193093291390288341638060014242859340626675116086733538306117755925947891767165625293420755946927417069994775515464067260600751272103349180234381869661841706395781821530533432601)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(0)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(41163154460832577483927594481740992203322996420492979926312215346678487095

True

In [13]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 5fc2a487-f351-45cb-95c3-04ab63f0c295
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(6298799670842520090584972555017294743610740008453085710738375356609898794867323040820452132027416932415727306453126353051300218869407523381609546145403216250174664837277155055914938581618764136283046297849652656950968134871258093753206694301271363461350536794014170269696688080865424128008543360323147891692382272409906872777412522655954056135541948138389770389507226528123571307983048555554259608036910626852647756884960282701619375543939466430105728246657400681707774198785510711938063614703103787649540309999390910890974396267819901159294651714380233915476463759778549768370505074798338300759639360675991176851878)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(0)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(516250643369965625422398994180963414940876436978551539480343959150097885318

False

---

## Протокол Шнорра
![Schnorr Protocol](./images/Schnorr_2.png)

In [14]:
class SchnorrCA(object):
    RSA_KEY_LENGTH = 4096
    
    def __init__(self):
        """
        Инициализация доверенного центра
        Генерация простых чисел p, q и элемента g, а также ключевой пары СА
        """
        
        self._db = {}
        
        #
        # Генерация параметров протокола
        #
        
        self._p, self._q, self._g = SchnorrCA._generate_parameters()
        
        #
        # Ключевая пара RSA для СА
        #
        
        self._private_key = RSA.generate(SchnorrCA.RSA_KEY_LENGTH)
        self._public_key = self._private_key.publickey()
        
        trace('[SchnorrCA]', f'RSA public key = {(self._public_key.e, self._public_key.n)}')
        
        
    def register_client(self, 
                        client_id: uuid.UUID, 
                        client_public_key: gmpy2.mpz) -> tuple[uuid.UUID, gmpy2.mpz, bytes]:
        """
        Регистрация клиента. Принимает на вход идентификатор клиента и его открытый ключ.
        Возвращает подписанный сервером сертификат.
        Выбрасывает исключение, если клиент уже зарегистрирован.
        """
        
        if client_id in self._db:
            trace('[SchnorrCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Создаю сертификат
        #
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        signature = pss.new(self._private_key).sign(h)
        
        certificate = (client_id, client_public_key, signature)
        trace('[SchnorrCA]', f'Certificate = {certificate}')
        
        #
        # Сохраняю его и возвращаю
        #
        
        self._db[client_id] = certificate
        return certificate
    
    
    def verify_certificate(self, certificate: tuple[uuid.UUID, gmpy2.mpz, bytes]) -> bool:
        """
        Проверка сертификата.
        Возвращает признак валидности сертификата.
        """
        
        client_id, client_public_key, signature = certificate
        
        if client_id not in self._db:
            trace('[SchnorrCA]', f"Client {client_id} doesn't exist")
            raise ValueError("Client doesn't exist")
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        verifier = pss.new(self.public_key)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа СА.
        """
        
        return self._public_key
    
    
    @property
    def parameters(self) -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Параметры протокола (p, q, g)
        """
        
        return self._p, self._q, self._g
        
        
    @staticmethod
    def _generate_parameters() -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Генерация параметров протокола (p, q, g).
        """
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = gmpy2.mpz(number.getPrime(Q_LENGTH))
        p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
        
        while not gmpy2.is_prime(p):
            p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
            
        #
        # Сгенерирую число, взаимно простое с p
        #
        
        tmp = getrandbits(P_LENGTH)
        while gmpy2.gcd(tmp, p) != 1:
            tmp = getrandbits(P_LENGTH)
        
        #
        # По малой теореме Ферма такой g будет иметь порядок q
        #
        
        g = gmpy2.powmod(tmp, (p - 1) // q, p)
        
        trace('[SchnorrCA]', f'{p = }, {q = }, {g = }')
        return p, q, g

In [15]:
class SchnorrClient(object):
    def __init__(self, parameters: tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz], *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz, tuple] = None):
        """
        Инициализация клиента. Генерирует идентификатор клиента,
        ключевую пару.
        Последний параметр необходим для демонстрации неуспешной аутентификации.
        """
        
        self._p, self._q, self._g = parameters
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        
        self._cert = None if fake_data is None else fake_data[2]
        self._r = None
        
        self._private_key = randint(1, self._q - 1)
        self._public_key  = gmpy2.powmod(
            self._g, -self._private_key, self._p) if fake_data is None else fake_data[1]
        
        trace('[SchnorrClient]', f'Client {self._id}')
        trace('[SchnorrClient]', f'Public key = {self._public_key}')
        
        
    def register(self, ca: SchnorrCA) -> None:
        """
        Регистрация клиента на СА. Принимает дескриптор СА.
        """
        
        self._cert = ca.register_client(self.identifier, self.public_key)
        
        
    def get_x(self) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = g ** r mod p
        где r - случайное целое число из отрезка [1, q - 1].
        Сохраняет значение r.
        """
        
        self._r = randint(1, self._q - 1)
        return gmpy2.powmod(self._g, self._r, self._p)
    
    
    def get_y(self, e, security_parameter) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = a * e + r mod q
        где r - сохраненное в get_x служебное значение, e - число,
        полученное от проверяющего, a - закрытый ключ клиента.
        """
        
        if e > 2 ** security_parameter or e < 1:
            trace('[SchnorrClient]', f'Invalid {e = }')
            raise ValueError('Invalid e')
            
        pre_result = gmpy2.mul(self._private_key, e) + self._r
        return gmpy2.t_mod(pre_result, self._q)
    
    
    @property
    def certificate(self):
        """
        Получение сертификата клиента.
        """
        
        return self._cert
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента.
        """
        
        return self._public_key
        
        
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id

In [16]:
class SchnorrVerifier(object):
    def __init__(self, parameters, security_parameter):
        """
        Инициализация проверяющего. Фактически только закрепляет параметры протокола.
        """
        
        self._p, self._q, self._g = parameters
        self._security_parameter = security_parameter
        
        assert security_parameter > 0 and 2 ** security_parameter < self._q
        
        self._e_values = set()
        
        
    def authenticate(self, client: SchnorrClient, ca: SchnorrCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        #
        # Получаю с клиента x и его сертификат
        #
        
        x, cert = client.get_x(), client.certificate
        trace('[SchnorrVerifier]', f'{x = }')
        
        #
        # Если сертификат битый или невалидный, то это плохо
        #
        
        if not ca.verify_certificate(cert):
            trace('[SchnorrVerifier]', f'Invalid certificate {cert}')
            return False
            
        #
        # Генерирую число е и отдаю его клиенту
        #
            
        e = randint(1, 2 ** self._security_parameter)
        while e in self._e_values:
            e = randint(1, 2 ** self._security_parameter)
            
        self._e_values.add(e)        
        trace('[SchnorrVerifier]', f'{e = }')
        
        y = client.get_y(e, self._security_parameter)
        trace('[SchnorrVerifier]', f'{y = }')
        
        #
        # Вычисляю значение, которое необходимо сравнить с полученным от клиента х
        # И сразу же сравниваю их
        # 
        
        product = gmpy2.mul(gmpy2.powmod(self._g, y, self._p), 
                            gmpy2.powmod(client.public_key, e, self._p))
        z = gmpy2.t_mod(product, self._p)
        trace('[SchnorrVerifier]', f'{z = }')
        
        return z == x

----

In [17]:
# Создаю доверенный центр
ca = SchnorrCA()

[TRACE] [SchnorrCA] p = mpz(21843055627881061533855993905068335366190476373292765009013677277199343736773077065428946022226517784459529940112254533842936978200170983623192087965078524259221583032082753222889085733905158166484011521671144111184617009292205595920455255289953165966816002541399294170210847131219300733691089449873620993759), q = mpz(1368590255070031019598797209154856682581870552979), g = mpz(4811773995078007022877454468942371638539066582946301856013246865184559309776519296823169214738560858370270939289948489133524892831518290820968060260542034760183261566413404824100307402730388299584554384433184411739152467542864943396218269311873521597530907788317927818635529076977347585610984595355474510409)
[TRACE] [SchnorrCA] RSA public key = (65537, 67951837559951189360935396858506821425680271769368589412370690722635625084460936483111469944621657965974377561341135424528151923867727069924247624838890828591825515936899834409026918591291392198803352736945356043435165693658836418887370115

In [18]:
# Создам Алису - валидного пользователя
alice = SchnorrClient(ca.parameters)

[TRACE] [SchnorrClient] Client 3205e579-a43e-444b-ab88-2a6ba7079878
[TRACE] [SchnorrClient] Public key = 11040610787771164140264605032679072778125183166780633211496940499298095620258398760738590176537387202756587178884029230920677152964536846618287821823423841795519392495889352127793070597419659393654389684722980055539418701872567347940382078124030058535784521383990221608211697097871692785559453094211220722834


In [19]:
# Регистрирую валидного пользователя
alice.register(ca)

[TRACE] [SchnorrCA] Certificate = (UUID('3205e579-a43e-444b-ab88-2a6ba7079878'), mpz(11040610787771164140264605032679072778125183166780633211496940499298095620258398760738590176537387202756587178884029230920677152964536846618287821823423841795519392495889352127793070597419659393654389684722980055539418701872567347940382078124030058535784521383990221608211697097871692785559453094211220722834), b'\x0c\x1c]\xa2\x9a1\xd3\x00r,-\xe5\x91\xbb\xf9!\xef\x9d\xef\x80z\x1b:Vx\x08"#E\xdc\xa5f\xf73\xb9\xfe\xe9w\xde\x1e\xe5\xe9\xa5\xb1\xec\xfa\x8d\xa7\xedV\x89tl\xe6X\xe6\xc38cn4\x11"\x1c7g\xa2fTz8\xc71d\xc1^\x1b\xf1\xb6{\x88\xd3\xf3\t\xab\x90\xfd\x14Z@\xdb\xd3\xb0\xa8\x1do\x05d`G\x1a\xb3\xe2\xa0\x85\x88\xa7\xeb0\xb8]\xab\x83AK\xdapn G\x0f\x84Y\xc9\xe0E\x1fm+\xd2n6\xfb\x96\xa6\'\xb3\xfd,\xf0\xcaG\xf2h\x83U\xea\xaf/\x0c\xb9\x98\x84K\xdc\xa6\x95\xc5&\xf4\xb3\xfc\xb6\xaf\xba\x14]\'\x9a\x1a\xecs\x05\xaf(\x9e\x9ceL\x98J\xc5\xca\xcf\xcc!\xda \xc3\x83\x99\xffS\x19\xd8\xdc;Px=\xaf\xd22\xf4\x16\x84T/A\x1c:_\r#

In [20]:
# Параметр безопасности положим равным 10 для демонстрации
security_parameter = 10

# Создаю проверяющую сторону
verifier = SchnorrVerifier(ca.parameters, security_parameter)

In [21]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [SchnorrVerifier] x = mpz(11795176220582379145026343947476400323166481279523997748835033851615544156365426936756379170883304011454292273438400195120531786710342014589012674970277714035591975874760418233313981309155793334419945434793783932622379351863839969587844685678185043855750791177802802159383517300106198010880703156419510941908)
[TRACE] [SchnorrVerifier] e = mpz(817)
[TRACE] [SchnorrVerifier] y = mpz(536386192056873017601376283530884127083462697614)
[TRACE] [SchnorrVerifier] z = mpz(11795176220582379145026343947476400323166481279523997748835033851615544156365426936756379170883304011454292273438400195120531786710342014589012674970277714035591975874760418233313981309155793334419945434793783932622379351863839969587844685678185043855750791177802802159383517300106198010880703156419510941908)


True

In [22]:
# Создаю Еву - клиента, который пытается представиться Алисой
eve = SchnorrClient(ca.parameters, 
                    fake_data=(alice.identifier, alice.public_key, alice.certificate))

[TRACE] [SchnorrClient] Client 3205e579-a43e-444b-ab88-2a6ba7079878
[TRACE] [SchnorrClient] Public key = 11040610787771164140264605032679072778125183166780633211496940499298095620258398760738590176537387202756587178884029230920677152964536846618287821823423841795519392495889352127793070597419659393654389684722980055539418701872567347940382078124030058535784521383990221608211697097871692785559453094211220722834


In [23]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [SchnorrVerifier] x = mpz(10046386103460235446380523081827918324924651569473675232910009433211152831314253016121144019587629177312868437065520098871249755618756707915841988766951600043385615089093735122224123731480119945156625006485625325579093252786801122526918243839888333188390750607994753334851222214949651436417199261784412604710)
[TRACE] [SchnorrVerifier] e = mpz(717)
[TRACE] [SchnorrVerifier] y = mpz(1097821162142371085909746378572521339578474624248)
[TRACE] [SchnorrVerifier] z = mpz(15232364513263369532873916665684223776779184903066382796017151197278762315163844004695233998336924559704720984906945215519701515620132702432955338584762711501213273323390592743170533867963249499540564743135938510963216571927292151651026145668305964839053893107468488897312337836138105297880901182522072065716)


False